In [2]:
# libraries
import random
from keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
nltk.download("wordnet")


# init file
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intents.json").read()
intents = json.loads(data_file)

# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# lemmatizer
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

# actual training
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

#Optional snippet - I have commented it as I did not use it.

# from keras import callbacks 
# earlystopping = callbacks.EarlyStopping(monitor ="loss", mode ="min", patience = 5, restore_best_weights = True)
# callbacks =[earlystopping]

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
44 documents
11 classes ['delivery', 'funny', 'goodbye', 'greeting', 'home remedies', 'information', 'items', 'payments', 'precautions', 'symptoms', 'thanks']
83 unique lemmatized words ["'s", '19', 'a', 'accept', 'against', 'anyone', 'are', 'at', 'be', 'bye', 'can', 'card', 'cash', 'common', 'covid', 'credit', 'day', 'delivery', 'detail', 'do', 'doe', 'efficient', 'extreme', 'fight', 'for', 'from', 'funny', 'get', 'good', 'goodbye', 'have', 'hello', 'helpful', 'hey', 'hi', 'home', 'how', 'i', 'info', 'information', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'more', 'my', 'of', 'on', 'only', 'pay', 'paypal', 'precaution', 'prevent', 'provide', 'remedy', 'remmedies', 'safe', 'see', 'sell', 'serious', 'shipping', 'something', '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


6/6 [==============================] - 1s 3ms/step - loss: 2.3648 - accuracy: 0.0359
Epoch 2/1000
6/6 [==============================] - 0s 2ms/step - loss: 2.3614 - accuracy: 0.0335
Epoch 3/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.3886 - accuracy: 0.0605
Epoch 4/1000
6/6 [==============================] - 0s 2ms/step - loss: 2.4188 - accuracy: 0.0955
Epoch 5/1000
6/6 [==============================] - 0s 2ms/step - loss: 2.3926 - accuracy: 0.0719
Epoch 6/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.4374 - accuracy: 0.0210
Epoch 7/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.4269 - accuracy: 0.0872
Epoch 8/1000
6/6 [==============================] - 0s 2ms/step - loss: 2.3344 - accuracy: 0.1134
Epoch 9/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.3541 - accuracy: 0.1530
Epoch 10/1000
6/6 [==============================] - 0s 3ms/step - loss: 2.3728 - accuracy: 0.1059
Epoch 11/1000
6/6 [=============